# Get license plate dataset

In [ ]:
!curl -L "https://universe.roboflow.com/ds/kdXoju18dM?key=7X5PeoFIWk" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
 extracting: train/xemay2040_jpg.rf.aab87edfce96e94eb21fda59097c1680.jpg  
 extracting: train/xemay2040_jpg.rf.f5715421bfb25ae3e3b3d20a72226064.jpg  
 extracting: train/xemay2041_jpg.rf.48933e36147e932feda2879c1fb39e95.jpg  
 extracting: train/xemay2041_jpg.rf.563e0fe10bd74e45aa02a7063e385513.jpg  
 extracting: train/xemay2041_jpg.rf.da93b949c97478ca5c3f3da82b98afca.jpg  
 extracting: train/xemay2047_jpg.rf.12bc3c6e45e8871fe522f7bdced0e2b3.jpg  
 extracting: train/xemay2047_jpg.rf.3f33fb2afdc0e15dd4f2b310ca62bc03.jpg  
 extracting: train/xemay2047_jpg.rf.468c278fb0b03d40bf0c2c2fe2b75631.jpg  
 extracting: train/xemay2048_jpg.rf.675d6a373e117f75c89281bc40f758be.jpg  
 extracting: train/xemay2048_jpg.rf.98feb087e3b03b296bc62267368fba61.jpg  
 extracting: train/xemay2048_jpg.rf.d6b163cc200b8041695dc258fa340d6b.jpg  
 extracting: train/xemay2049_jpg.rf.79f7feab13ddb732fecea2624ef3d651.jpg  
 extracting: train/xemay2049_jpg.rf

In [ ]:
!mkdir license_plate_data; mv train ./license_plate_data/train; mv test ./license_plate_data/test; mv valid ./license_plate_data/valid;

Create models directory for keeping the best model so far

In [ ]:
!mkdir models

mkdir: cannot create directory ‘models’: File exists


# DATASET CLASS

In [ ]:
from torch.utils.data import Dataset
from torchvision.io.image import read_image
import json
import cv2
import numpy as np
import torch
class LicensePlateDataset(Dataset):

  def __init__(self,  mode,annotation_path):
    self.base_path = f"{annotation_path}/{mode}/"
    f = open(self.base_path+"_annotations.coco.json")
    data = json.load(f)
    self.images = data["images"]
    self.annotations = data["annotations"]

  def __len__(self):

    return len(self.images)

  def __getitem__(self, idx):

    input_idx = self.annotations[idx]["image_id"]
    input, label = self.get_input(input_idx)
    output = np.array([label, *self.get_output(idx)]).astype(np.float32).reshape(1,-1)
    return input, str({
        "labels": [label],
        "boxes": [self.get_output(idx)]
    })

  def get_input(self, idx):

    img_path = self.base_path + self.images[idx]["file_name"]
    img = read_image(img_path)

    label = self.images[idx]["license"]

    return img, label


  def get_output(self, idx):
    box_corrs = self.annotations[idx]["bbox"]
    return [box_corrs[0], box_corrs[1], box_corrs[0]+box_corrs[2], box_corrs[1]+box_corrs[3]]


# Utils

In [ ]:
from typing import Tuple, List, Dict, Optional
import torch
from torch import Tensor
from collections import OrderedDict
from torchvision.models.detection.roi_heads import fastrcnn_loss
from torchvision.models.detection.rpn import concat_box_prediction_layers
def eval_forward(model, images, targets):
    """
    Args:
        images (list[Tensor]): images to be processed
        targets (list[Dict[str, Tensor]]): ground-truth boxes present in the image (optional)
    Returns:
        result (list[BoxList] or dict[Tensor]): the output from the model.
            It returns list[BoxList] contains additional fields
            like `scores`, `labels` and `mask` (for Mask R-CNN models).
    """
    model.eval()

    original_image_sizes: List[Tuple[int, int]] = []
    for img in images:
        val = img.shape[-2:]
        assert len(val) == 2
        original_image_sizes.append((val[0], val[1]))

    images, targets = model.transform(images, targets)

    # Check for degenerate boxes
    if targets is not None:
        for target_idx, target in enumerate(targets):
            boxes = target["boxes"]
            degenerate_boxes = boxes[:, 2:] <= boxes[:, :2]
            if degenerate_boxes.any():
                # print the first degenerate box
                bb_idx = torch.where(degenerate_boxes.any(dim=1))[0][0]
                degen_bb: List[float] = boxes[bb_idx].tolist()
                raise ValueError(
                    "All bounding boxes should have positive height and width."
                    f" Found invalid box {degen_bb} for target at index {target_idx}."
                )

    features = model.backbone(images.tensors)
    if isinstance(features, torch.Tensor):
        features = OrderedDict([("0", features)])
    model.rpn.training=True


    features_rpn = list(features.values())
    objectness, pred_bbox_deltas = model.rpn.head(features_rpn)
    anchors = model.rpn.anchor_generator(images, features_rpn)

    num_images = len(anchors)
    num_anchors_per_level_shape_tensors = [o[0].shape for o in objectness]
    num_anchors_per_level = [s[0] * s[1] * s[2] for s in num_anchors_per_level_shape_tensors]
    objectness, pred_bbox_deltas = concat_box_prediction_layers(objectness, pred_bbox_deltas)

    # apply pred_bbox_deltas to anchors to obtain the decoded proposals
    # the proposals
    proposals = model.rpn.box_coder.decode(pred_bbox_deltas.detach(), anchors)
    proposals = proposals.view(num_images, -1, 4)
    proposals, scores = model.rpn.filter_proposals(proposals, objectness, images.image_sizes, num_anchors_per_level)

    proposal_losses = {}
    assert targets is not None
    labels, matched_gt_boxes = model.rpn.assign_targets_to_anchors(anchors, targets)
    regression_targets = model.rpn.box_coder.encode(matched_gt_boxes, anchors)
    loss_objectness, loss_rpn_box_reg = model.rpn.compute_loss(
        objectness, pred_bbox_deltas, labels, regression_targets
    )
    proposal_losses = {
        "loss_objectness": loss_objectness,
        "loss_rpn_box_reg": loss_rpn_box_reg,
    }

    image_shapes = images.image_sizes
    proposals, matched_idxs, labels, regression_targets = model.roi_heads.select_training_samples(proposals, targets)
    box_features = model.roi_heads.box_roi_pool(features, proposals, image_shapes)
    box_features = model.roi_heads.box_head(box_features)
    class_logits, box_regression = model.roi_heads.box_predictor(box_features)

    result: List[Dict[str, torch.Tensor]] = []
    detector_losses = {}
    loss_classifier, loss_box_reg = fastrcnn_loss(class_logits, box_regression, labels, regression_targets)
    detector_losses = {"loss_classifier": loss_classifier, "loss_box_reg": loss_box_reg}
    boxes, scores, labels = model.roi_heads.postprocess_detections(class_logits, box_regression, proposals, image_shapes)
    num_images = len(boxes)
    for i in range(num_images):
        result.append(
            {
                "boxes": boxes[i],
                "labels": labels[i],
                "scores": scores[i],
            }
        )
    detections = result
    detections = model.transform.postprocess(detections, images.image_sizes, original_image_sizes)  # type: ignore[operator]
    model.rpn.training=False
    model.roi_heads.training=False
    losses = {}
    losses.update(detector_losses)
    losses.update(proposal_losses)
    return losses, detections

In [ ]:
import ast

def train(n_steps, model, optimizer, device, epoch, train_loader, preprocess, batch_size):
  model.train()
  losses_l = []

  for stp, batch in enumerate(train_loader):
    optimizer.zero_grad()

    images = list(preprocess(image).to(device) for image in batch[0])
    targets = [ast.literal_eval(target) for target in batch[1]]
    targets = [{k: torch.tensor(v).to(device) for k,v in target.items()} for target in targets]

    losses = model(images, targets)
    loss = sum(loss for loss in losses.values())
    losses_l.append(loss.item())
    loss.backward()
    optimizer.step()
    print('train | epoch = {}, iter = [{}|{}], loss = {}, loss_box = {}'.format(epoch, stp, n_steps,
                                                                                round(loss.item(), 6),
                                                                                round(losses['loss_box_reg'].item(), 6)))
    if stp > n_steps:
      break

  return np.mean(losses_l)



def validate(n_steps, model, optimizer, device, epoch, val_loader, preprocess, batch_size):
  losses_l = []
  for stp, batch in enumerate(val_loader):
    images = list(preprocess(image).to(device) for image in batch[0])
    targets = [ast.literal_eval(target) for target in batch[1]]
    targets = [{k: torch.tensor(v).to(device) for k,v in target.items()} for target in targets]

    losses, detections = eval_forward(model, images,targets)
    loss = sum(loss for loss in losses.values())
    losses_l.append(loss.item())

    print('val | epoch = {}, iter = [{}|{}], loss = {},  loss_box = {}'.format(epoch, stp, n_steps,
                                                                                round(loss.item(), 6),
                                                                                round(losses['loss_box_reg'].item(), 6)))

  return np.mean(losses_l)


# CONFIG

In [ ]:
BATCH_SIZE = 16
NUM_EPOCHS = 500

STEPS_PER_EPOCH = 200
device = 'cuda'

LEARNING_RATE = 0.005
MOMENTUM = 0.9
WEIGHT_DECAY = 0.0005

EXPORT_PATH = "./models/best_model.pt"
last_best = None

# TRAINING

In [ ]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2,FasterRCNN_ResNet50_FPN_V2_Weights
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import ast
weights = FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT

preprocess = weights.transforms()


train_dataset = LicensePlateDataset("train", "./license_plate_data")
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    pin_memory=True
    )



val_dataset = LicensePlateDataset("valid", "./license_plate_data")
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    pin_memory=True
    )




model = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.7)

# disable gradient flow
for layer in model.parameters():
  layer.requires_grad = False

# create new predicting layer that will be trained
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)
model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)


for epoch in range(NUM_EPOCHS):
  train_loss = train(STEPS_PER_EPOCH, model, optimizer, device, epoch, train_loader, preprocess, BATCH_SIZE)
  val_loss = validate(STEPS_PER_EPOCH, model, optimizer, device, epoch, val_loader, preprocess, BATCH_SIZE)
  lr_scheduler.step()
  print("TRAIN LOSS: {}, VALIDATION LOSS: {}".format(train_loss.item(), val_loss.item()))
  if last_best is None or last_best > val_loss.item():
    last_best = val_loss.item()
    torch.save(model, EXPORT_PATH)

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
train | epoch = 14, iter = [178|200], loss = 0.143994, loss_box = 0.020067
train | epoch = 14, iter = [179|200], loss = 0.09333, loss_box = 0.012361
train | epoch = 14, iter = [180|200], loss = 0.111669, loss_box = 0.018332
train | epoch = 14, iter = [181|200], loss = 0.151364, loss_box = 0.011737
train | epoch = 14, iter = [182|200], loss = 0.157224, loss_box = 0.017336
train | epoch = 14, iter = [183|200], loss = 0.140156, loss_box = 0.021789
train | epoch = 14, iter = [184|200], loss = 0.095665, loss_box = 0.020971
train | epoch = 14, iter = [185|200], loss = 0.120472, loss_box = 0.026087
train | epoch = 14, iter = [186|200], loss = 0.108458, loss_box = 0.013063
train | epoch = 14, iter = [187|200], loss = 0.135512, loss_box = 0.01585
train | epoch = 14, iter = [188|200], loss = 0.11107, loss_box = 0.006705
train | epoch = 14, iter = [189|200], loss = 0.120106, loss_box = 0.017903
train | epoch = 14, iter = [190|200], 

KeyboardInterrupt: ignored